# Data Quality con Great Expectations

In [1]:
# ! pip install great_expectations

In [2]:
import os
import sys
import pandas as pd
import datetime
import re
from ruamel import yaml
from ruamel.yaml import YAML
sys.path.append('../../')

## Ge
import great_expectations as gx
import great_expectations as gx
import great_expectations.jupyter_ux
from great_expectations.cli.datasource import sanitize_yaml_and_save_datasource, check_if_datasource_name_exists
from great_expectations.core.batch import RuntimeBatchRequest
from great_expectations.core.expectation_configuration import ExpectationConfiguration
from great_expectations.data_context.types.resource_identifiers import ExpectationSuiteIdentifier
from great_expectations.exceptions import DataContextError
from great_expectations.checkpoint.types.checkpoint_result import CheckpointResult


from pprint import pprint




/opt/conda/lib/python3.10/inspect.py:469: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  value = getattr(object, key)
/opt/conda/lib/python3.10/inspect.py:469: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  value = getattr(object, key)
/opt/conda/lib/python3.10/inspect.py:469: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  value = getattr(object, key)


2023-02-08T19:55:20+0000 - INFO - Great Expectations logging enabled at 20 level by JupyterUX module.


# Data Quality 

Dopo aver completato la fase di Data Ingestion tramite lo scraping il secondo step è quello di valutare la qualità dei dati raccolti.  
Il dataframe studiato presenta 18 colonne. Ogni colonna presenta delle differenze dalle altre e ci si aspetta determinati valori da queste.

Dopo aver settato:
1. la datasource
2. suite (e le relative expectations)
3. il checkpoint

Si condurrà una validazione del sample

In [3]:
for e in os.listdir('../data'):
    print(e)

scraping_all_amsterdam_company_template.csv
scraping_all_amsterdam_job_template.csv
scraping_all_amsterdam_template.csv
scraping_all_londra_080223.csv
scraping_all_londra_company_080223.csv
scraping_all_londra_job_080223.csv


In [4]:
df = pd.read_csv('../data/scraping_all_londra_080223.csv')

In [5]:
list(df.columns)

['Unnamed: 0',
 'company',
 'job_title',
 'location',
 'company_rating',
 'job_age',
 'job_link',
 'Opportunità di carriera',
 'Stipendio e benefit',
 'Cultura e valori',
 'Dirigenti senior',
 'Equilibrio lavoro/vita privata',
 'Sede centrale',
 'Dimensioni',
 'Fondata nel',
 'Tipo',
 'Settore',
 'Segmento',
 'Entrate']

## Istruzioni operative
E' necessario dichiare il data_asset_name e la expectation_suite_name che si vuole utilizzare

In [6]:
# inserisci qua il nome del file senza .csv
data_asset_name = 'scraping_all_londra_080223'

In [7]:
expectation_suite_name = 'Main'

In [8]:
context = gx.get_context()

2023-02-08T19:55:20+0000 - INFO - FileDataContext loading zep config
2023-02-08T19:55:20+0000 - INFO - GxConfig.parse_yaml() failed with errors - [{'loc': ('xdatasources',), 'msg': 'field required', 'type': 'value_error.missing'}]
2023-02-08T19:55:20+0000 - INFO - GxConfig.parse_yaml() returning empty `xdatasources`
2023-02-08T19:55:20+0000 - INFO - Loading 'datasources' ->
{}
2023-02-08T19:55:20+0000 - INFO - Loaded 'datasources' ->
{}
2023-02-08T19:55:21+0000 - INFO - Usage statistics is disabled; skipping initialization.


## Configurazione della datasource 

In [9]:
datasource_config: dict = {
    "name": "glassdoor_scraping",
    "class_name": "Datasource",
    "module_name": "great_expectations.datasource",
    "execution_engine": {
        "class_name": "PandasExecutionEngine",
        "module_name": "great_expectations.execution_engine",
    },
     "data_connectors": {
        "all": {
            "class_name": "InferredAssetFilesystemDataConnector",
            "base_directory": "../data",
            "default_regex": {"pattern": "(.*)\\.csv",
            "group_names": ["data_asset_name"]},
            #"batch_spec_passthrough": {
            #    "reader_method": "read_csv",
            #    "reader_options": {
            #        "header": True,
            #        "inferSchema": True,
            #    },
            #},
        }
    },
}

In [10]:
#check configurazione
context.test_yaml_config(yaml.dump(datasource_config))

Attempting to instantiate class from config...
	Instantiating as a Datasource, since class_name is Datasource
	Successfully instantiated Datasource


ExecutionEngine class name: PandasExecutionEngine
Data Connectors:
	all : InferredAssetFilesystemDataConnector

	Available data_asset_names (3 of 6):
		scraping_all_amsterdam_company_template (1 of 1): ['scraping_all_amsterdam_company_template.csv']
		scraping_all_amsterdam_job_template (1 of 1): ['scraping_all_amsterdam_job_template.csv']
		scraping_all_amsterdam_template (1 of 1): ['scraping_all_amsterdam_template.csv']

	Unmatched data_references (0 of 0):[]



In [11]:
## try to add
try:
    context.get_datasource(datasource_config["name"])
except ValueError:
    context.add_datasource(**datasource_config)
else:
    print(
        f"The datasource {datasource_config['name']} already exists in your Data Context!"
    )

The datasource glassdoor_scraping already exists in your Data Context!


In [12]:
# context.add_datasource(**datasource_config) # usa questo comando per sovrascrive la data source 

In [13]:
context.list_datasources()

[{'data_connectors': {'all': {'class_name': 'InferredAssetFilesystemDataConnector',
    'module_name': 'great_expectations.datasource.data_connector',
    'default_regex': {'group_names': ['data_asset_name'],
     'pattern': '(.*)\\.csv'},
    'base_directory': '../data'}},
  'name': 'glassdoor_scraping',
  'class_name': 'Datasource',
  'module_name': 'great_expectations.datasource',
  'execution_engine': {'class_name': 'PandasExecutionEngine',
   'module_name': 'great_expectations.execution_engine'}}]

## Creazione della Suite
Dopo aver configurato la data source creaimo le suite e le aspettive

In [14]:
context = gx.get_context()


# Feel free to change the name of your suite here. Renaming this will not remove the other one.
#expectation_suite_name = "Main"
try:
    suite = context.get_expectation_suite(expectation_suite_name=expectation_suite_name)
    print(f'Loaded ExpectationSuite "{suite.expectation_suite_name}" containing {len(suite.expectations)} expectations.')
except DataContextError:
    suite = context.create_expectation_suite(expectation_suite_name=expectation_suite_name)
    print(f'Created ExpectationSuite "{suite.expectation_suite_name}".')

2023-02-08T19:55:21+0000 - INFO - FileDataContext loading zep config
2023-02-08T19:55:21+0000 - INFO - GxConfig.parse_yaml() failed with errors - [{'loc': ('xdatasources',), 'msg': 'field required', 'type': 'value_error.missing'}]
2023-02-08T19:55:21+0000 - INFO - GxConfig.parse_yaml() returning empty `xdatasources`
2023-02-08T19:55:21+0000 - INFO - Loading 'datasources' ->
{}
2023-02-08T19:55:21+0000 - INFO - Loaded 'datasources' ->
{}
2023-02-08T19:55:21+0000 - INFO - Usage statistics is disabled; skipping initialization.
Loaded ExpectationSuite "Main" containing 4 expectations.


### 1 Table Expectation

#### Questa Expectation verifica che le colonne nella tabella abbiano questo ordine
- company
- job_title
- location
- company_rating
- job_age
- job_link
- Sedeentrale'
- Dimensioni
- Fondatael'
- Tipo
- Entrate
- Opportunitài carriera'
- Stipendio benefit'
- Cultura valori'
- Dirigentienior'
- Equilibrioavoro/vita privata'
- Settore
- Segmento

In [15]:
# Create an Expectation
expectation_configuration = ExpectationConfiguration(
   # Name of expectation type being added
   expectation_type="expect_table_columns_to_match_ordered_list",
   # These are the arguments of the expectation
   # The keys allowed in the dictionary are Parameters and
   # Keyword Arguments of this Expectation Type
   kwargs={
      "column_list": [
                    'company',
                    'job_title',
                    'location',
                    'company_rating',
                    'job_age',
                    'job_link',
                    'Sede centrale',
                    'Dimensioni',
                    'Fondata nel',
                    'Tipo',
                    'Entrate',
                    'Opportunità di carriera',
                    'Stipendio e benefit',
                    'Cultura e valori',
                    'Dirigenti senior',
                    'Equilibrio lavoro/vita privata',
                    'Settore',
                    'Segmento']
   },
   # This is how you can optionally add a comment about this expectation.
   # It will be rendered in Data Docs.
   # See this guide for details:
   # `How to add comments to Expectations and display them in Data Docs`.
   meta={
      "notes": {
         "format": "markdown",
         "content": ""
      }
   }
)
# Add the Expectation to the suite
suite.add_expectation(expectation_configuration=expectation_configuration)

{"meta": {"notes": {"format": "markdown", "content": ""}}, "expectation_type": "expect_table_columns_to_match_ordered_list", "kwargs": {"column_list": ["company", "job_title", "location", "company_rating", "job_age", "job_link", "Sede centrale", "Dimensioni", "Fondata nel", "Tipo", "Entrate", "Opportunit\u00e0 di carriera", "Stipendio e benefit", "Cultura e valori", "Dirigenti senior", "Equilibrio lavoro/vita privata", "Settore", "Segmento"]}}

### 2. Column Expectation

#### Company

##### company  
    Questa Expectation verifica che la colonna company sia di tipo stringa

In [16]:
expectation_configuration = ExpectationConfiguration(
   expectation_type="expect_column_values_to_be_in_type_list",
   kwargs={
        "column": "company",
      "type_list": ['str'],
   },
   # Note optional comments omitted
)
suite.add_expectation(expectation_configuration=expectation_configuration)

{"meta": {}, "expectation_type": "expect_column_values_to_be_in_type_list", "kwargs": {"column": "company", "type_list": ["str"]}}

##### company  
    Questa Expectation verifica che la colonna company non sia MAI nulla

In [17]:
expectation_configuration = ExpectationConfiguration(
   expectation_type="expect_column_values_to_not_be_null",
   kwargs={
      "column": "company",
   },
   # Note optional comments omitted
)
suite.add_expectation(expectation_configuration=expectation_configuration)

{"meta": {}, "expectation_type": "expect_column_values_to_not_be_null", "kwargs": {"column": "company"}}

##### company  
    Questa Expectation verifica che la colonna company non sia MAI valorizzata con i numeri che vanno da 1 a 10

In [18]:
expectation_configuration = ExpectationConfiguration(
   expectation_type="expect_column_values_to_not_be_in_set",
   kwargs={
      "column": "company",
      "value_set": [1,2,3,4,5,6,7,8,9,10]
   },
   # Note optional comments omitted
)
suite.add_expectation(expectation_configuration=expectation_configuration)

{"meta": {}, "expectation_type": "expect_column_values_to_not_be_in_set", "kwargs": {"column": "company", "value_set": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}}

### Salvataggio delle suite e creazione della documentazione
rimuovere il commento dalla prima riga per la costruzione della suite

In [19]:
# print(context.get_expectation_suite(expectation_suite_name=expectation_suite_name))
context.save_expectation_suite(expectation_suite=suite, expectation_suite_name=expectation_suite_name)

suite_identifier = ExpectationSuiteIdentifier(expectation_suite_name=expectation_suite_name)
context.build_data_docs(resource_identifiers=[suite_identifier])
context.open_data_docs(resource_identifier=suite_identifier)

### Apri questo link per vedere le expectations create

In [20]:
print('http://localhost:9000/view/great_expectations/uncommitted/data_docs/local_site/expectations/'+ expectation_suite_name +'.html')

http://localhost:9000/view/great_expectations/uncommitted/data_docs/local_site/expectations/Main.html


Se non si sta utilizzando il notebook dal docker si può arrivare al sito tramite il percorso 'great_expectations\uncommitted\data_docs\local_site'

## Creazione di un checkpoint

In [21]:
## checkpoint
yaml = YAML()

In [22]:
my_checkpoint_name = "main_checkpoint" # This was populated from your CLI command.

yaml_config = f"""
name: {my_checkpoint_name}
config_version: 1.0
class_name: SimpleCheckpoint
run_name_template: "%Y%m%d-%H%M%S-{data_asset_name}"
validations:
  - batch_request:
      datasource_name: glassdoor_scraping 
      data_connector_name: all
      data_asset_name: {data_asset_name}
      data_connector_query:
        index: -1
    expectation_suite_name: Main
"""

In [23]:
# Run this cell to print out the names of your Datasources, Data Connectors and Data Assets
pprint(context.get_available_data_asset_names())

{'glassdoor_scraping': {'all': ['scraping_all_londra_job_080223',
                                'scraping_all_londra_080223',
                                'scraping_all_amsterdam_job_template',
                                'scraping_all_londra_company_080223',
                                'scraping_all_amsterdam_company_template',
                                'scraping_all_amsterdam_template']}}


In [24]:
context.list_expectation_suite_names()

['Main']

In [25]:
main_checkpoint = context.test_yaml_config(yaml_config=yaml_config)

Attempting to instantiate class from config...
	Instantiating as a SimpleCheckpoint, since class_name is SimpleCheckpoint
	Successfully instantiated SimpleCheckpoint


Checkpoint class name: SimpleCheckpoint


In [26]:
#print(main_checkpoint.get_config(mode="yaml"))

In [27]:
# aggiungere un checkpoint
# se il checkpoint è già presente verrà sovrascritto
context.add_checkpoint(**yaml.load(yaml_config))
context.list_checkpoints()

['main_checkpoint']

[Come aggiungere dati o suite di convalida a un Checkpoint](https://docs.greatexpectations.io/docs/guides/validation/checkpoints/how_to_add_validations_data_or_suites_to_a_checkpoint

In [28]:
try:
    results = context.run_checkpoint(
        checkpoint_name="main_checkpoint",
        expectation_suite_name = 'Main'
    )
except:
    print('ko')

2023-02-08T19:55:22+0000 - INFO - 	4 expectation(s) included in expectation_suite.


Calculating Metrics:   0%|          | 0/18 [00:00<?, ?it/s]

In [29]:
# A questo punto la validazione è completata, puoi consultare i risultati

In [30]:
a = re.findall('[0-9]{8}-[0-9]{6}-' + data_asset_name + '/[0-9]{8}T[0-9]{6}.[0-9]{6}Z/[\S]{32}', str(list(results['run_results'])[0]))[0]
b = expectation_suite_name + '/' + a
path = r"../great_expectations/uncommitted/validations/" + b + '.json'

In [31]:
## potrebbe essere interessante parsarlo in un dataframe per avere i risultati come dati tabellari
import json

with open(path, "r") as read_file:
    data = json.load(read_file)

print(data)

{'evaluation_parameters': {}, 'meta': {'active_batch_definition': {'batch_identifiers': {}, 'data_asset_name': 'scraping_all_londra_080223', 'data_connector_name': 'all', 'datasource_name': 'glassdoor_scraping'}, 'batch_markers': {'ge_load_time': '20230208T195522.349144Z', 'pandas_data_fingerprint': 'd88db22d02c578e9c3a63872d9728ed6'}, 'batch_spec': {'path': '/home/jovyan/great_expectations/../data/scraping_all_londra_080223.csv'}, 'checkpoint_id': None, 'checkpoint_name': 'main_checkpoint', 'expectation_suite_name': 'Main', 'great_expectations_version': '0.15.47', 'run_id': {'run_name': '20230208-195522-scraping_all_londra_080223', 'run_time': '2023-02-08T19:55:22.324469+00:00'}, 'validation_id': None, 'validation_time': '20230208T195522.376969Z'}, 'results': [{'exception_info': {'exception_message': None, 'exception_traceback': None, 'raised_exception': False}, 'expectation_config': {'expectation_type': 'expect_table_columns_to_match_ordered_list', 'kwargs': {'batch_id': '97d75456846

### Apri questo link per vedere le expectations create

In [32]:
print('http://localhost:9000/view/great_expectations/uncommitted/data_docs/local_site/expectations/'+ expectation_suite_name +'.html')

http://localhost:9000/view/great_expectations/uncommitted/data_docs/local_site/expectations/Main.html


### Se non stai usando docker vai in questo path 

In [33]:
print("great_expectations/uncommitted/data_docs/local_site/expectations/"+ expectation_suite_name +'.html')

great_expectations/uncommitted/data_docs/local_site/expectations/Main.html


### Apri questo link per vedere i risultati


In [34]:
print( "http://localhost:9000/view/great_expectations/uncommitted/data_docs/local_site/validations/" + b + '.html')

http://localhost:9000/view/great_expectations/uncommitted/data_docs/local_site/validations/Main/20230208-195522-scraping_all_londra_080223/20230208T195522.324469Z/97d7545684674b50fe8cbe7c68cd9ad0.html


### Se non stai usando docker vai in questo path 

In [35]:
print("great_expectations/uncommitted/data_docs/local_site/validations/" + b + '.html')

great_expectations/uncommitted/data_docs/local_site/validations/Main/20230208-195522-scraping_all_londra_080223/20230208T195522.324469Z/97d7545684674b50fe8cbe7c68cd9ad0.html
